# DIC EX2 - part 2

## Setup

### Initialize Spark context

In [28]:
from pyspark.ml.feature import (
    RegexTokenizer, StopWordsRemover,
    CountVectorizer, IDF, ChiSqSelector, StringIndexer
)

In [29]:
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .appName("DIC EX 2 - group 36") \
    .getOrCreate()

### Set path variables

In [30]:
SEED = 42
DEV_JSON = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"         
SAVE_PATH = "feature_pipe_part2"

### Load data

In [32]:
df = spark.read.json("reviews_devset.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



## Build pipeline

### Tokenize using regex

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

#Create Regex Tokenizer Instance
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="tokens", pattern="[\s\t\d\(\)\[\]\{\}\.\!\?\,\;\:\+\=\-\_\"\'`\~\#\@\&\*\%\€\$\§\\\/]+")

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\timgr\AppData\Local\Temp\ipykernel_32304\3300093208.py:5: SyntaxWarning: invalid escape sequence '\s'
  tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="tokens", pattern="[\s\t\d\(\)\[\]\{\}\.\!\?\,\;\:\+\=\-\_\"\'`\~\#\@\&\*\%\€\$\§\\\/]+")


### Remove stopwords

In [ ]:
from pyspark.ml.feature import StopWordsRemover

#Create method to load stopword file
def load_stopwords(path= "stopwords.txt") -> list[str]:
    """
    Load stopwords from a file efficiently.
    """
    stopwords = set()
    with open(path, "r", encoding="utf-8") as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return list(stopwords)


#Create StopWordsRemover Instance using our provided Stopwords file loaded with the function above
stopper = StopWordsRemover(
    inputCol="tokens", outputCol="tokens_filt",
    stopWords=load_stopwords()
)

### Calculate token counts and idf

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer

#Instantiate CountVectorizer required for the chi-squared calculation
tf = CountVectorizer(
    inputCol="tokens_filt", outputCol="tf",
    vocabSize=20_000, minDF=5
)


#Create IDF instance
idf = IDF(inputCol="tf", outputCol="tf_idf")


In [ ]:
#index category
label_indexer = StringIndexer(
    inputCol="category", outputCol="label", handleInvalid="skip"
)

### Calculate chi square values and select top 75 features

In [ ]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType



#now we create the chi square selector instance
chisq = ChiSqSelector(
    featuresCol="tf_idf", outputCol="selected_features",
    labelCol="label",        
    numTopFeatures=2_000
)



In [ ]:
from pyspark.ml import Pipeline, PipelineModel

#finally, we combine all previously created instances in a feature_pipe
feature_pipe = Pipeline(stages=[
    tokenizer, stopper, tf, idf, label_indexer, chisq
])

print("Pipeline running")
#we then use the pipe and fit it on the data set.
feature_model = feature_pipe.fit(df)



Pipeline running


In [ ]:
from pyspark.ml import PipelineModel
import getpass

#safe to hdfs so we can use it later in part 3 without creating it again.
USER = getpass.getuser()                  
SAVE_PATH = f"hdfs:///user/{USER}/models/feature_pipe_part2"

feature_model.write().overwrite().save(SAVE_PATH)

### Get top tokens

In [ ]:
from pyspark.ml.feature import CountVectorizerModel, ChiSqSelectorModel

#now we extract the countvectorizermodel again from the fitted pipeline stages, so we can get the vocabulary then
cv_model = next(
    s for s in feature_model.stages
    if isinstance(s, CountVectorizerModel)
)
vocab = cv_model.vocabulary

#same for the chi-squared selector, we extract it from the fitted pipeline stages, so we then get all the selected features from the fitted chi square selector
sel_model = next(s for s in feature_model.stages
                 if isinstance(s, ChiSqSelectorModel))
all_selected = sel_model.selectedFeatures    

#extract top 2000 tokens from the selected features in the chi-squared selector
top2000_tokens = [vocab[i] for i in all_selected]

#write to file
with open("output_ds.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(top2000_tokens))

25/04/26 22:00:09 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


['amazon', 'author', 'back', 'bad', 'big', 'bit', 'bought', 'buy', 'character', 'characters', 'day', 'easy', 'end', 'enjoyed', 'excellent', 'family', 'feel', 'find', 'fit', 'found', 'give', 'good', 'great', 'happy', 'hard', 'high', 'highly', 'interesting', 'job', 'light', 'long', 'lot', 'love', 'loved', 'made', 'make', 'makes', 'man', 'money', 'music', 'nice', 'part', 'people', 'perfect', 'pretty', 'price', 'problem', 'purchase', 'purchased', 'put', 'quality', 'quot', 'reading', 'real', 'recommend', 'review', 'series', 'set', 'size', 'small', 'sound', 'thing', 'things', 'thought', 'time', 'times', 'wanted', 'watch', 'work', 'works', 'world', 'worth', 'written', 'year', 'years']
